In [1]:
import pandas as pd
import requests
import re

In [26]:
for i in range(50):
    try:
        with open (str(i)+'.txt','r',encoding='utf-8') as topic, open('quiz.txt','a') as file:
            to = topic.read()
            to = to.split('\n')
            for t in to:
                try:
                    word = t.split(',')[0]
                    if word != '':
                        file.write(word+'\n')
                except UnicodeEncodeError:
                    pass
    except FileNotFoundError:
        pass

In [64]:
to_text=[]
with open ('topic.txt','r',encoding='utf-8') as file:
    to = file.read()
    to = to.split('\n')
    for w in to:
        to_text.append(w)

In [65]:
with open('testdic.txt','r',encoding='utf-8') as file:
    to = file.read()
    to = to.split('\n')
    for w in to_text:
        if w in to:
            to_text.remove(w)

In [69]:
with open('testdic.txt','a',encoding='utf-8') as file:
    for w in to_text:
        file.write(w+'\n')

In [71]:
to_final=[]
with open('testdic.txt','r',encoding='utf-8') as file:
    to = file.read()
    to = to.split('\n')
    for w in to:
        to_final.append(w)

In [73]:
to_final = to_final[1:]

In [75]:
to_final.append('感情')

In [77]:
with open('final.txt','a',encoding='utf-8') as file:
    for w in to_final:
        file.write(w+'\n')

In [349]:
import time
import re

In [366]:
def crowl_content(id):
    url = 'https://www.dcard.tw/_api/posts/'+str(id)
    web = requests.get(url)
    jsn = web.json()
    if len(jsn)>5:
        if len(jsn['content'])>10 and len(jsn['title'])>3:
            content = jsn['content']
            title = jsn['title']
            topics = jsn['topics']
            output = content+title
            for w in topics:
                output+=(w+'\n')
            for_del = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',output)
            for d in for_del:
                output= output.replace(d,' ')
            return output
        else:
            pass
    else:
        pass
    return 'no_content'


In [299]:
content_list = []
for i in range(230000000,240000000):
    print(i)
    try:
        content = crowl_content(i)
        if content:
            content_list.append(content)
    except KeyError:
        pass

In [375]:
url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'
web = requests.get(url)
jsn = web.json()

content_list = []

for i in range(len(jsn)):

    ids = jsn[i]['id']
    content_list.append(crowl_content(ids))
    last = ids
    
for j in range(100):
    url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + str(last)
    web = requests.get(url)
    jsn = web.json()
    if len(jsn)>1:
        for i in range(len(jsn)):
            ids = jsn[i]['id']
            try:
                content_list.append(crowl_content(ids))
                last = ids
            except KeyError:
                last = ids


In [368]:
len(content_list)

1674

In [369]:
None in content_list

False

In [311]:
while None in content_list:
    content_list.remove(None)

In [312]:
None in content_list

False

In [313]:
len(content_list)

1940

In [81]:
import jieba
import pandas as pd
from collections import Counter
import math

In [82]:
jieba.load_userdict('final.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\88691\AppData\Local\Temp\jieba.cache
Loading model cost 0.778 seconds.
Prefix dict has been built successfully.


In [314]:
def find_keywords(article_content):
    
    def keyword(sentence):
    #     cut the sentence with jieba
        article = sentence
        article_cut = jieba.lcut(article)
    # only take words that length > 2
        filter_cut = []
        for w in article_cut:
            if len(w)>1:
                filter_cut.append(w)
    # count 'tf' values
        c = Counter(filter_cut)
        total = len(set(filter_cut))
        df = pd.DataFrame(c.keys(),c.values())
        df['tf'] = df.index/total


    # def log function
        def log(x):
            return math.log(x,10)

        total_content = len(content_list)
    # count this word be metions in other articles times
        metion_list=[]
        for word in df[0]:
            metion = 0
            for a in content_list:
                if word in a:
                    metion+=1
            if metion == 0:
                metion+=1
            metion_list.append(metion)
    # count 'idf' values
        df['metion']=metion_list
        df['idf'] = df['metion'].apply(lambda x : log(total_content/x))
    # count 'tfidf' values
        df['tfidf'] = df['tf']*df['idf']

        df = df.sort_values('tfidf',ascending=False)
        return df

    kw = keyword(article_content)
    # weight the topic
    topic_word = []
    with open('topic.txt','r',encoding='utf-8') as file:
        for w in file:
            w = w.strip()
            topic_word.append(w)

    c = Counter(topic_word)
    kw['score'] = kw[0].apply(lambda x : c[x] if x in topic_word else 0)
    
    kw['weight'] = kw['score'].apply(lambda x : 1+2*x)
    kw['output'] = kw['tfidf']*kw['weight']

    pd.options.display.max_rows=50000
    kw.columns=['topic', 'tf', 'metion', 'idf', 'tfidf', 'score', 'weight', 'output']

    kw = kw.sort_values('output',ascending=False)

    return kw

In [315]:
article = '''因應武漢肺炎禁出口口罩一個月😷

會禁止出口是有道理的


直接上圖

黑字是某單位阿長傳的內容
（傳給學妹所在的單位，學妹截圖發在限時動態，我經過她同意再截圖下來）
粉紅底白字是我想說的




下圖是我朋友在超商的經歷
又剛好遇到某藝人的發言
我整個都要爆氣了





真的希望可以把資源留給需要的人⋯⋯
'''

In [316]:
find_keywords(article)

,topic,tf,metion,idf,tfidf,score,weight,output
1,武漢肺炎,0.023256,225,0.935619,0.021759,74,149,3.242029
1,口罩,0.023256,254,0.882968,0.020534,10,21,0.431217
1,超商,0.023256,32,1.782652,0.041457,4,9,0.373113
1,限時動態,0.023256,14,2.141674,0.049806,1,3,0.149419
2,學妹,0.046512,6,2.509650,0.116728,0,1,0.116728
2,單位,0.046512,21,1.965582,0.091422,0,1,0.091422
2,截圖,0.046512,31,1.796440,0.083555,0,1,0.083555
1,長傳,0.023256,1,3.287802,0.076461,0,1,0.076461
1,黑字,0.023256,1,3.287802,0.076461,0,1,0.076461
2,出口,0.046512,60,1.509650,0.070216,0,1,0.070216


In [321]:
find_keywords('''
喬科維奇 vs 費德勒 比賽討論區
歡迎大家一起來討論這場世界前三名的對決吧！
幫各自支持的球員加油

''')

,topic,tf,metion,idf,tfidf,score,weight,output
1,喬科維奇,0.058824,1,3.287802,0.193400,1,3,0.580200
1,費德勒,0.058824,1,3.287802,0.193400,1,3,0.580200
1,前三名,0.058824,1,3.287802,0.193400,0,1,0.193400
1,討論區,0.058824,2,2.986772,0.175692,0,1,0.175692
1,球員,0.058824,3,2.810680,0.165334,0,1,0.165334
1,對決,0.058824,5,2.588832,0.152284,0,1,0.152284
1,比賽,0.058824,11,2.246409,0.132142,0,1,0.132142
1,這場,0.058824,11,2.246409,0.132142,0,1,0.132142
1,vs,0.058824,12,2.208620,0.129919,0,1,0.129919
1,各自,0.058824,21,1.965582,0.115622,0,1,0.115622


In [322]:
find_keywords('''安妞～
這五支是我平常出場率最高的唇彩
顏色都很日常百搭 也可以說是我的無腦唇彩😂
不知道要擦什麼或懶得想的時候就會選它們

先來張合照👇🏻

分得出來誰是誰嗎🤣🤣🤣


2019下半年開始莫名很愛木質玫瑰這種色調
就是玫瑰色再多一點棕色調或濁色調
不像以往的玫瑰色偏粉
-
▪️ Loreal #小鋼筆129
薄擦是很正的玫瑰色 厚塗顏色會變深一點更沈穩
小鋼筆的質地之輕薄就是讚
收了四支顏色都很喜歡 擦這支很常被問哈哈


▪️ heme #柔霧慕斯唇釉 #02
質感媲美專櫃的開架唇彩
質地比小鋼筆稍有存在感但一樣也很舒服不緊繃
這兩支顏色有點像 同系列的04我也很喜歡～


▪️ Visee Avant Lipstick#006
在版上一出來馬上被燒到的磚紅色
整體的質感從包裝到質地也很棒 顯白不挑膚色
蠻難找的後來在屈臣氏就沒看到了🤔


▪️ Loreal #純色訂製唇膏柔霧款 #298
這支可以說是出場率冠軍🏆
溫柔知性 帶濁色調就算顏色掉了也不會很明顯
蠻意外它在同系列中相對討論度沒那麼高🤔
淡妝想要稍微提氣色的話很適合 上課最常帶它


▪️ 露華濃 #經典璀璨唇膏 #535
秋冬感豆沙紅棕色
有點紫色調可甜可鹽但不顯老氣
薄擦氣質 厚塗有氣勢 顏色在開架也蠻少見的

-
以上.
Btw 大家出門到公共場所記得戴口罩嘿
👋🏻''')

,topic,tf,metion,idf,tfidf,score,weight,output
2,唇膏,0.014599,31,1.796440,0.026225,25,51,1.337496
3,唇彩,0.021898,14,2.141674,0.046898,4,9,0.422082
1,露華濃,0.007299,1,3.287802,0.023999,4,9,0.215987
1,唇釉,0.007299,10,2.287802,0.016699,5,11,0.183692
1,heme,0.007299,8,2.384712,0.017407,4,9,0.156660
1,屈臣氏,0.007299,8,2.384712,0.017407,3,7,0.121847
3,小鋼筆,0.021898,1,3.287802,0.071996,0,1,0.071996
4,色調,0.029197,8,2.384712,0.069627,0,1,0.069627
6,顏色,0.043796,59,1.516950,0.066436,0,1,0.066436
1,Visee,0.007299,2,2.986772,0.021801,1,3,0.065404


In [335]:
crowl_content(232983452)

'想請教各位\n因為買單趟到加拿大台灣出發的機票都蠻貴\n於是我搜尋菲律賓出發 \n如截圖所示 菲律賓-台灣-加拿大\n我可以放棄前段 直接使用後段check-in嗎\nhttps://megapx-assets.dcard.tw/images/30679a87-4dc7-454e-a357-0fcf0ce4d3ff/1280.jpeg機票購買詢問機票\n詢問\n'

In [357]:
find_keywords(crowl_content(232983452)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
2,台灣,0.083333,242,0.903986,0.075332,16,33,2.485963
2,菲律賓,0.083333,7,2.442704,0.203559,3,7,1.424910
3,機票,0.125000,16,2.083682,0.260460,2,5,1.302301
2,加拿大,0.083333,16,2.083682,0.173640,2,5,0.868201
2,出發,0.083333,25,1.889862,0.157488,0,1,0.157488


In [358]:
find_keywords(crowl_content(232983455)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
5,筆電,0.068493,15,2.111710,0.144638,9,19,2.748116
1,電腦,0.013699,37,1.719600,0.023556,8,17,0.400455
2,醫院,0.027397,46,1.625044,0.044522,3,7,0.311652
4,ASUS,0.054795,2,2.986772,0.163659,0,1,0.163659
1,文書,0.013699,6,2.509650,0.034379,1,3,0.103136


In [346]:
find_keywords(crowl_content(232983464)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
1,春聯,0.017241,2,2.986772,0.051496,4,9,0.463465
1,理膚寶水,0.017241,4,2.685742,0.046306,3,7,0.324141
1,momo,0.017241,5,2.588832,0.044635,2,5,0.223175
1,試用包,0.017241,4,2.685742,0.046306,1,3,0.138918
2,gu,0.034483,342,0.753776,0.025992,2,5,0.129961


In [360]:
find_keywords(crowl_content(232983492)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
25,口罩,0.109649,254,0.882968,0.096817,30,61,5.905817
9,洗手,0.039474,36,1.731499,0.068349,2,5,0.341743
2,臉書,0.008772,33,1.769288,0.015520,6,13,0.201761
4,醫院,0.017544,46,1.625044,0.028510,3,7,0.199567
1,假新聞,0.004386,3,2.810680,0.012328,6,13,0.160258


In [351]:
find_keywords(crowl_content(232983501)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
2,醫生,0.037037,49,1.597606,0.059171,2,5,0.295853
2,感冒,0.037037,18,2.032529,0.075279,1,3,0.225837
1,小孩,0.018519,54,1.555408,0.028804,2,5,0.144019
2,的藥,0.037037,14,2.141674,0.079321,0,1,0.079321
4,一樣,0.074074,270,0.856438,0.063440,0,1,0.063440


In [352]:
find_keywords(crowl_content(232983506)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
4,太妍,0.034783,6,2.509650,0.087292,4,9,0.785630
2,lv,0.017391,6,2.509650,0.043646,7,15,0.654691
2,台灣,0.017391,242,0.903986,0.015722,15,31,0.487367
2,首爾,0.017391,8,2.384712,0.041473,5,11,0.456206
4,EXO,0.034783,2,2.986772,0.103888,1,3,0.311663


In [353]:
find_keywords(crowl_content(232983522)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
3,口罩,0.053571,254,0.882968,0.047302,10,21,0.993339
1,台灣,0.017857,242,0.903986,0.016143,15,31,0.500421
3,台北,0.053571,65,1.474888,0.079012,2,5,0.395059
1,酸民,0.017857,6,2.509650,0.044815,1,3,0.134446
2,五千,0.035714,4,2.685742,0.095919,0,1,0.095919


In [354]:
find_keywords(crowl_content(232983523)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
1,蝦皮,0.012658,32,1.782652,0.022565,37,75,1.692391
2,粉刺,0.025316,8,2.384712,0.060372,7,15,0.905587
1,痘痘,0.012658,11,2.246409,0.028436,4,9,0.255920
4,水楊酸,0.050633,1,3.287802,0.166471,0,1,0.166471
3,寶拉,0.037975,1,3.287802,0.124853,0,1,0.124853


In [355]:
find_keywords(crowl_content(232983536)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
4,台中,0.023529,37,1.719600,0.040461,2,5,0.202306
3,壽司,0.017647,3,2.810680,0.049600,1,3,0.148801
1,instagram,0.005882,43,1.654333,0.009731,7,15,0.145971
3,沙拉,0.017647,5,2.588832,0.045685,1,3,0.137056
3,美食,0.017647,21,1.965582,0.034687,1,3,0.104060


In [356]:
find_keywords(crowl_content(232983538)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
3,台灣,0.057692,242,0.903986,0.052153,15,31,1.616745
3,網路霸凌,0.057692,5,2.588832,0.149356,2,5,0.746778
1,口罩,0.019231,254,0.882968,0.016980,10,21,0.356583
2,酸民,0.038462,6,2.509650,0.096525,1,3,0.289575
2,范瑋琪,0.038462,25,1.889862,0.072687,1,3,0.218061


In [362]:
find_keywords(crowl_content(232983538)).head(5)

,topic,tf,metion,idf,tfidf,score,weight,output
3,台灣,0.057692,242,0.903986,0.052153,16,33,1.721051
2,范瑋琪,0.038462,25,1.889862,0.072687,7,15,1.090305
1,口罩,0.019231,254,0.882968,0.016980,30,61,1.035789
3,網路霸凌,0.057692,5,2.588832,0.149356,2,5,0.746778
2,酸民,0.038462,6,2.509650,0.096525,1,3,0.289575


In [373]:
with open('all_dcard_articles.txt','a',encoding='utf-8') as file:
    for ar in content_list:
        file.write(ar+'\n')